In [1]:
import pandas as pd
df = pd.read_csv('train.csv')
print(df.head())

                                          image_link  group_id  entity_name  \
0  https://m.media-amazon.com/images/I/61I9XdN6OF...    748919  item_weight   
1  https://m.media-amazon.com/images/I/71gSRbyXmo...    916768  item_volume   
2  https://m.media-amazon.com/images/I/61BZ4zrjZX...    459516  item_weight   
3  https://m.media-amazon.com/images/I/612mrlqiI4...    459516  item_weight   
4  https://m.media-amazon.com/images/I/617Tl40LOX...    731432  item_weight   

     entity_value  
0      500.0 gram  
1         1.0 cup  
2      0.709 gram  
3      0.709 gram  
4  1400 milligram  


In [2]:
import os
import requests
from PIL import Image, ImageFile
import io

ImageFile.LOAD_TRUNCATED_IMAGES = True

# Function to fetch image from a link and open it with PIL
def fetch_image(link):
    try:
        response = requests.get(link, timeout=10)
        response.raise_for_status()  # Check for a valid response
        img = Image.open(io.BytesIO(response.content))
        return img
    except Exception as e:
        print(f"Error fetching image: {e}")
        return None


In [3]:
import os

def process_images(df, download_folder, limit=10):
    # Ensure the folder to save images exists
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    
    # Limit the number of images to process
    for idx, image_link in enumerate(df['image_link'][:limit]):
        # Fetch image
        image = fetch_image(image_link)
        
        if image:
            # Save image locally (optional)
            file_name = f"image_{idx}.jpg"
            file_path = os.path.join(download_folder, file_name)
            image.save(file_path)  # Save image as JPEG
            print(f"Image {idx} saved: {file_name}")
        else:
            print(f"Failed to process image {idx}")

# Call the function with a limit of 2000 images
process_images(df, 'Images/')

Error fetching image: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process image 0
Error fetching image: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process image 1
Error fetching image: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process image 2
Error fetching image: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process image 3
Error fetching image: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
Failed to process image 4
Error fetching image: ('Connection aborted.', ConnectionResetError(10054, '

In [4]:
import os
print(os.getcwd())  # Print the current working directory

c:\Users\hp\OneDrive\Documents\GitHub\AmazonMLhack


In [5]:
import pytesseract

# Print the current value of tesseract_cmd
print(pytesseract.pytesseract.tesseract_cmd)

tesseract


In [6]:
import pytesseract
from PIL import Image

# Specify the correct path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_image(image_path):
    with Image.open(image_path) as img:
        text = pytesseract.image_to_string(img)
    return text

# Ensure the image path is correct
image_path = r'C:\Users\hp\OneDrive\Documents\GitHub\AmazonMLhack\Images\image_0.jpg'

# Extract text from the image
text = extract_text_from_image(image_path)
print("Extracted Text:")
print(text)


Extracted Text:
100% NATUR




In [11]:
import os
import pytesseract
from PIL import Image

# Specify the correct path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_image(image_path):
    with Image.open(image_path) as img:
        text = pytesseract.image_to_string(img)
    return text

def extract_text_from_specific_image(folder_path, image_name):
    # Construct the full path for the specific image
    image_path = os.path.join(folder_path, image_name)
    
    if os.path.exists(image_path):
        # Extract text from the image
        text = extract_text_from_image(image_path)
        print(f"Extracted Text from {image_name}:")
        print(text)
    else:
        print(f"Image not found: {image_path}")

# Now, extract text from a specific image
specific_image_name = 'image_2.jpg'  # Change this to the image you want to process
extract_text_from_specific_image('Images/', specific_image_name)


Extracted Text from image_2.jpg:
Serving Size: 1 Tablet (0.709 g) | Each serving contains (Approx. Values):

Ingredient Oty, / Serving

*PHOSPHOcomplex® Silybin (Sillybum marianum) 200 mg
Dandelion (Taraxacum officinale) leaf extract - 10:1 100 mg
Kutki (Picrorhiza kurroa)rhizome extract - 0.5% Bitters 50 mg
Kasani (Cichorium intybus) seed extract - 1%. Bitters 25 mg
Punarnava (Boerhavia diffusa) root extract - 0.07% alkaloids 25 mg
Bhui amla (Phyllanthus amarus) WP extract - 0.5% Bitters 25 mg
Amla (Phyllanthus emblica) fruit extract - 10% Tannins 25 mg
Licorice (Glycyrrhiza glabra) root extract - 5% Glycyrrhizin 25 mg
Vitamin E 10 mg
Piper nigrum fruit extract — 95% Piperine 5mg

Nutrients Qty. / Serving

Energy 3.04 kcal
Carbohydrate 051g
(Sugars) 02g
Protein 0.049
Fat 0.099

"ZRDA values established as per ICMR 2010 for sedentary lifestyle-Men.
**Z RDA not established by ICMR




In [9]:
import re

def extract_value_from_text(text, entity_name):
    # Convert the text to lower case for case-insensitive search
    text_lower = text.lower()
    entity_name_lower = entity_name.lower()
    
    # Create a regex pattern to match the entity_name and its value
    pattern = re.compile(rf'{re.escape(entity_name_lower)}\s*[:\-]*\s*(\d+\.?\d*)\s*(\w*)', re.IGNORECASE)
    
    # Search for the pattern in the text
    match = pattern.search(text_lower)
    
    if match:
        value = match.group(1)
        unit = match.group(2)
        return f"{value} {unit}" if unit else value
    else:
        return "Value not found"
